# Example of using RDFPandas with Gastrodon

## Intro
We would ideally want a roundtrip from Pandas DataFrame to RDF and back without loss of types, which is possible in theory. The plan is to evolve rdfpandas to cover all of these cases in include SPARQL magics support for Jupyter.

## Example

In [9]:
import rdfpandas.graph
import pandas as pd
import rdflib
import requests

In [10]:
df = pd.DataFrame(data = {
        'http://xmlns.com/foaf/0.1/firstName': ['Vicki', 'Brian'], 
        'http://xmlns.com/foaf/0.1/lastName': ['Mason', 'Watts'], 
        'http://xmlns.com/foaf/0.1/age': [38, 36],
        }, 
        index=['http://www.semanticweblondon.com/1', 'http://www.semanticweblondon.com/2'])
df


,http://xmlns.com/foaf/0.1/age,http://xmlns.com/foaf/0.1/firstName,http://xmlns.com/foaf/0.1/lastName
http://www.semanticweblondon.com/1,38,Vicki,Mason
http://www.semanticweblondon.com/2,36,Brian,Watts


In [11]:
graph = rdfpandas.graph.to_graph(df)

s = graph.serialize(format='turtle')
s

b'@prefix ns1: <http://xmlns.com/foaf/0.1/> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xml: <http://www.w3.org/XML/1998/namespace> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<http://www.semanticweblondon.com/1> ns1:age 38 ;\n    ns1:firstName "Vicki" ;\n    ns1:lastName "Mason" .\n\n<http://www.semanticweblondon.com/2> ns1:age 36 ;\n    ns1:firstName "Brian" ;\n    ns1:lastName "Watts" .\n\n'

In [12]:
url = 'http://rdf-jupyter_fuseki_1:3030/swl/data?default'
files = {'default.ttl': ('default.ttl', s, 'text/turtle')}
r = requests.request(method='PUT', url=url, files=files)
r.text

'{ \n  "count" : 6 ,\n  "tripleCount" : 6 ,\n  "quadCount" : 0\n}\n\r\n'

In [13]:
import sys
from os.path import expanduser
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
import pandas as pd
import json
pd.options.display.width=120
pd.options.display.max_colwidth=100

In [14]:
connection_data = json.loads(
"""
{
    "url":"http://rdf-jupyter_fuseki_1:3030/swl/sparql",
    "user":"admin",
    "passwd":"password",
    "base_uri":"http://www.semanticweblondon.com/"
}
""")

In [15]:
endpoint=RemoteEndpoint(**connection_data)

In [16]:
df = endpoint.select("""
select ?s ?p ?o 
where
{
   ?s ?p ?o .
}
""")
df

,s,p,o
0,http://www.semanticweblondon.com/1,http://xmlns.com/foaf/0.1/age,38
1,http://www.semanticweblondon.com/1,http://xmlns.com/foaf/0.1/firstName,Vicki
2,http://www.semanticweblondon.com/1,http://xmlns.com/foaf/0.1/lastName,Mason
3,http://www.semanticweblondon.com/2,http://xmlns.com/foaf/0.1/age,36
4,http://www.semanticweblondon.com/2,http://xmlns.com/foaf/0.1/firstName,Brian
5,http://www.semanticweblondon.com/2,http://xmlns.com/foaf/0.1/lastName,Watts
